### Introduction
We create a time series data at yearly resolution for each PRIO GRID cell with the development stage of SNL mining on that PRIO GRID cell.

1. Each PRIO GRID 44*4 rows information with 1 development stage information in each quarter in each year. Here are the year range: [1970, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
       1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022] and 4 quarter range: [Q1, Q2, Q3, Q4]
              
       The SNL company doesn't provide that data between 1970-1980.

2. If one PRIO GRID cell has multiple development stage(which means multiple SNL minings) in the same year, keep the latest stage. 

3. If there is no available data for that year, we use the latest development stage from previous year.

In [1]:
import numpy as np
import pandas as pd

### Read Prio Grid cell data

In [2]:
spine = pd.read_csv("../input/PRIO GRID spine.csv")
print(spine.shape)
spine.head()

(64818, 3)


gid    lon    lat
0  49182 -69.25 -55.75
1  49183 -68.75 -55.75
2  49184 -68.25 -55.75
3  49185 -67.75 -55.75
4  49186 -67.25 -55.75

### Read SNL data with SNL property ID and PRIO grid ID

In [3]:
df_SNL= pd.read_csv("../input/SNL-PRIO GRID IDs.csv")
print(df_SNL.shape)
df_SNL.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
df_SNL = df_SNL[pd.to_numeric(df_SNL['PRIO GRID ID'], errors='coerce').notnull()]
df_SNL["PRIO GRID ID"] = df_SNL["PRIO GRID ID"].astype("int")
df_SNL = df_SNL[(df_SNL['LATITUDE']<=90) & (df_SNL['LATITUDE']>= -90) &
                (df_SNL['LONGITUDE']<=180) & (df_SNL['LONGITUDE']>= -180)].reset_index().drop(columns="index",axis=1)
print(df_SNL.shape)
df_SNL.head()

(39612, 46)
(39370, 46)


/var/folders/0p/mkzn2l513fl51bl6rs96d9z40000gn/T/ipykernel_18793/2275177157.py:1: DtypeWarning: Columns (2,22,23,24,25,26,27,28,30,31,32,33,36,37,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_SNL= pd.read_csv("../input/SNL-PRIO GRID IDs.csv")


Unnamed: 0           Property name Property ID Also Known As  \
0          0           11 RAU-Novaya       68289           NaN   
1          1           12 Mile Creek       66178           NaN   
2          2               14 Karats       65525           NaN   
3          3  14 Mile Well Extension       63665           NaN   
4          4     15 Mile Silver City       65760           NaN   

                                     OWNER(S) Equity ownership %  \
0  Sodeistvie Private Enterprise (Owner) 100%                100   
1               U&D Coal Limited (Owner) 100%                100   
2          Dios Exploration Inc. (Owner) 100%                100   
3               Private Interest (Owner) 100%                100   
4       North Lily Mining Company (Owner) 50%                 50   

  DEVELOPMENT STAGE ACTIVITY STATUS   COMMODITY(S) PRIMARY COMMODITY  ...  \
0        Grassroots        Inactive           Coal              Coal  ...   
1        Grassroots        Inactive           Coal              Coal  ...   
2        Grassroots        Inactive  Gold,Diamonds              Gold  ...   
3        Grassroots        Inactive           Gold              Gold  ...   
4        Grassroots        Inactive    Gold,Silver              Gold  ...   

  Unnamed: 36 Unnamed: 37 Unnamed: 38 Unnamed: 39 Unnamed: 40  Unnamed: 41  \
0         NaN         NaN         NaN         NaN         NaN          NaN   
1         NaN         NaN         NaN         NaN         NaN          NaN   
2         NaN         NaN         NaN         NaN         NaN          NaN   
3         NaN         NaN         NaN         NaN         NaN          NaN   
4         NaN         NaN         NaN         NaN         NaN          NaN   

   Unnamed: 42 Unnamed: 43 Unnamed: 44 PRIO GRID ID  
0          NaN         NaN         NaN       199878  
1          NaN         NaN         NaN        96419  
2          NaN         NaN         NaN       205418  
3          NaN         NaN         NaN        88445  
4          NaN         NaN         NaN       221841  

[5 rows x 46 columns]

In [4]:
subset_df = df_SNL[["PRIO GRID ID", "Property ID"]]
subset_df.head()

PRIO GRID ID Property ID
0        199878       68289
1         96419       66178
2        205418       65525
3         88445       63665
4        221841       65760

### Read SNL data with SNL property ID with development stage and source date

In [5]:
def process(path):
    tmp = pd.read_excel(path)
    print(tmp.shape)
    tmp = tmp.drop([0,1,2,4,5], axis=0)
    tmp.head()
    df = tmp.iloc[1:,:]
    df.columns = tmp.iloc[0]
    return df

In [6]:
work_full_history = process('../input/development stage with quarter.xls')
print(work_full_history.shape)
work_full_history.head()

(39854, 5)
(39848, 5)


3                  PROP_NAME PROP_ID    DEV_STAGE YR_SOURCE_DATE  \
6         Chihuahua Refinery   86959    Operating           2020   
7                        101   64665    Operating           2011   
8                       1068   79522  Exploration           2020   
9   10th of Ramadan Refinery   86452    Operating           2021   
10   10th of Ramadan Smelter   86451    Operating           2021   

3  MO_SOURCE_DATE  
6               2  
7              12  
8              10  
9               2  
10              2

In [7]:
work_full_history["PROP_ID"] =work_full_history["PROP_ID"].astype("int32")
subset_df["Property ID"] =subset_df["Property ID"].astype("int32")

/var/folders/0p/mkzn2l513fl51bl6rs96d9z40000gn/T/ipykernel_18793/3781885345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["Property ID"] =subset_df["Property ID"].astype("int32")


### Merge 2 SNL datasets based on SNL ID

In [8]:
subset_merge= subset_df.merge(work_full_history, left_on="Property ID",right_on="PROP_ID", how="inner")
subset_merge=subset_merge.drop(columns=["PROP_NAME", "PROP_ID"])
print(subset_merge.shape)
subset_merge.head()


(39354, 5)


PRIO GRID ID  Property ID   DEV_STAGE YR_SOURCE_DATE MO_SOURCE_DATE
0        199878        68289  Grassroots           2014              7
1         96419        66178  Grassroots           2014              2
2        205418        65525  Grassroots           2013              4
3         88445        63665  Grassroots           2013             12
4        221841        65760  Grassroots           2013              6

### Building the time-series data


Assign a numerical value to each development stage according to the order of development stage. Thus, we can compare the squential order of development stage according to the rules wrote in introduction aprt.

The larger the value is assigned, the stage is at the later step. For example, "grassroot" is the beginning stage, "Closed" is the last stage. 

Here is the explanation of each development stage:
https://upenn.app.box.com/file/1032975091991z


In [9]:
subset_merge = subset_merge.dropna()
print(subset_merge.shape)

# assign value to the development stage
subset_merge["DEV_STAGE"] =subset_merge["DEV_STAGE"].replace({
            # np.nan:0, 
            'Grassroots':1, 'Exploration':2,  'Advanced Exploration':3,
            'Target Outline':4, 'Reserves Development':5, 'Advanced Exploration':6, 'Prefeas/Scoping':7,
            'Feasibility':8,  'Feasibility Started':9,'Feasibility Complete':10, 
             'Preproduction': 11, 'Construction Planned':12, 'Construction Started':13, 
             'Commissioning':14, 'Operating':15, 'Satellite':16, 
            'Expansion':17, 'Limited Production':18, 'Residual Production':19, 'Closed':20
})

# assign value to the quarter
subset_merge["quarter"] = subset_merge["MO_SOURCE_DATE"].replace({
            # np.nan:0, 
            1:1, 2:1, 3:1,
            4:2, 5:2, 6:2, 
            7:3, 8:3, 9:3,
            10:4, 11:4, 12:4,
            })           

(36677, 5)


We only considerate SNL mining which is still not closed in 1990.

In [10]:
subset_drop_year = subset_merge.loc[~((subset_merge["DEV_STAGE"]=="Closed")&(subset_merge["YR_SOURCE_DATE"]<1990))]
subset_drop_year.head()

PRIO GRID ID  Property ID  DEV_STAGE YR_SOURCE_DATE MO_SOURCE_DATE  quarter
0        199878        68289          1           2014              7        3
1         96419        66178          1           2014              2        1
2        205418        65525          1           2013              4        2
3         88445        63665          1           2013             12        4
4        221841        65760          1           2013              6        2

In [11]:
year_ls = [1970, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
       1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
quarter_ls = [1,2,3,4]       

Categorize the SNL mining by PRIO GRID id and its time (quarter+year)

If there is only one SNL in that category, the development stage of PRIO GRID cell is the only-one development stage

If there are multiple SNL with multiple development stage in that category, the development stage of PRIO GRID cell is the latest development stage with largest assigned value

In [12]:
ls_prio_id = []
year = []
quarter = []
count = []
development_stage = []

# categorize the data by its PRIO GRID id, date year, date quarter
# count the snl property count in each category 
tmp_df = subset_drop_year.groupby(['PRIO GRID ID','YR_SOURCE_DATE',"quarter"])[["Property ID","DEV_STAGE"]]

for name_of_group, contents_of_group in tmp_df:
    ls_prio_id.append(name_of_group[0])
    year.append(name_of_group[1])
    quarter.append(name_of_group[2])
    count.append(len(contents_of_group))

    # if there is multiple development stage, choose the latest stage
    if len(contents_of_group) == 1:
        tmp = contents_of_group["DEV_STAGE"].to_list()[0]
    if len(contents_of_group)>1:
        tmp = 0 
        for i in contents_of_group["DEV_STAGE"].to_list():
            if i > tmp:
                tmp = i
    development_stage.append(tmp)


In [13]:
df_quarter = pd.DataFrame({"prio grid id":ls_prio_id,"year":year, "quarter":quarter, "count":count, "development_stage":development_stage })
df_quarter

prio grid id  year  quarter  count  development_stage
0             50619  1999        2      1                 20
1             50624  1998        4      1                  4
2             52063  1994        1      1                 15
3             52776  2022        3      1                  6
4             52778  2019        1      1                  4
...             ...   ...      ...    ...                ...
31426        238456  2005        3      1                  2
31427        242111  2006        3      1                  2
31428        243570  2010        2      1                  1
31429        244272  2010        1      1                  1
31430        244991  2014        4      1                  5

[31431 rows x 5 columns]

In [14]:
# expand the GRIO GRID id with each year in year_ls and each quarter in quarter ls
gid =[]
year_2 =[]
quarter_2 = []
def expand_function(x):
    for i in year_ls:
        for j in quarter_ls:
            gid.append(x)
            year_2.append(i)
            quarter_2.append(j)


In [15]:
spine["gid"].apply(lambda x :expand_function(x))

0        None
1        None
2        None
3        None
4        None
         ... 
64813    None
64814    None
64815    None
64816    None
64817    None
Name: gid, Length: 64818, dtype: object

In [16]:
new_spine = pd.DataFrame({"gid":gid, "year":year_2, "quarter":quarter_2})
print(new_spine.shape)

(11407968, 3)


In [17]:
# if there is corresponidng value in df_quarter, spine_dev will be filled by its
# if there is no value, spine_dev will be filled by 0
spine_dev = new_spine.merge(df_quarter, left_on=["gid","year", "quarter"], right_on=["prio grid id","year", "quarter"], how="left").fillna(0)
spine_dev["year"] = spine_dev["year"].astype("int")
spine_dev["quarter"] = spine_dev["quarter"].astype("int")
spine_dev["count"] = spine_dev["count"].astype("int")
spine_dev["development_stage"] = spine_dev["development_stage"].astype("int")

In [18]:
# transfer the df to dictionary with prio grid cell id, year, quarter as keys
dict ={}
def form_dict(id, year, quarter, count, development_stage_value):
    dict[(int(id),int(year),int(quarter))] = (count, development_stage_value)

In [19]:
spine_dev[["gid","year","quarter","count","development_stage"]].apply(lambda x: form_dict(*x), axis=1)

0           None
1           None
2           None
3           None
4           None
            ... 
11407963    None
11407964    None
11407965    None
11407966    None
11407967    None
Length: 11407968, dtype: object

In [20]:
gid_3 = []
year_3 =[]
quarter_3 = []
count_3 =[]
lateset_year_3 = []

# the function fill the later development stage without data  
# with previous lateset developement stage
def find_each_latest_year(gid, year, quarter, count, stage):
    gid_3.append(gid)
    year_3.append(year)
    quarter_3.append(quarter)

    if count ==0 or stage == 0:
        tmp_year = year_ls.index(year)
        tmp_quarter = quarter_ls.index(quarter)
        
        # if the date is 1970 Q1, and there is no data, 
        # set count and development stage as 0
        # todo: change if the first year!=1970
        if (year == 1970) and (quarter == 1):
            count_3.append(0)
            lateset_year_3.append(0)
        
        # if the data is 1970 Q2-Q4, and there is no data
        if year == 1970 and quarter!=1:
            # check the development stage of previous quarters at 1970
            for q in quarter_ls[:tmp_quarter][::-1]:
                if dict[(gid, year, q)][0]!=0 and dict[(gid, year, q)][1]!=0:
                    count_3.append(dict[(gid, year, q)][0])
                    lateset_year_3.append(dict[(gid, year, q)][1])
                    break
                if q == 1:
                    count_3.append(0)
                    lateset_year_3.append(0)   

        # if the data is 1980+ Q1, and there is no data
        if year != 1970 and quarter == 1:
            force_break_loop = False
            # check the development stage of previous quarters at previous year
            for y in year_ls[:tmp_year][::-1]:
                # and each quarter
                for q in quarter_ls[::-1]:
                    if dict[(gid, y, q)][0]!=0 and dict[(gid, y, q)][1]!=0:
                        count_3.append(dict[(gid, y, q)][0])
                        lateset_year_3.append(dict[(gid, y, q)][1])
                        force_break_loop = True
                        break
                    if y == 1970 and q == 1:
                        count_3.append(0)
                        lateset_year_3.append(0)   
                if force_break_loop:
                    break
                                
        # if the data is 1980+ Q2-Q4, and there is no data
        if year != 1970 and quarter !=1:  
            # check previous quarters at that year
            for q in quarter_ls[:tmp_quarter][::-1]:
                if dict[(gid, year, q)][0]!=0 and dict[(gid, year, q)][1]!=0:
                    count_3.append(dict[(gid, year, q)][0])
                    lateset_year_3.append(dict[(gid, year, q)][1])
                    break
                if q ==1 :
                    force_break_loop = False
                    # check previous years
                    for y in year_ls[:tmp_year][::-1]:
                        # check each quarters
                        for q_2 in quarter_ls[::-1]:
                            if dict[(gid, y, q_2)][0]!=0 and dict[(gid, y, q_2)][1]!=0:
                                count_3.append(dict[(gid, y, q_2)][0])
                                lateset_year_3.append(dict[(gid, y, q_2)][1])
                                force_break_loop = True
                                break
                            if y == 1970 and q_2 ==1:
                                count_3.append(0)
                                lateset_year_3.append(0)   
                        if force_break_loop:
                            break
    else:
        count_3.append(count)
        lateset_year_3.append(stage)            
        

In [21]:
spine_dev[["gid","year","quarter","count","development_stage"]].apply(lambda x: find_each_latest_year(*x), axis=1)

0           None
1           None
2           None
3           None
4           None
            ... 
11407963    None
11407964    None
11407965    None
11407966    None
11407967    None
Length: 11407968, dtype: object

In [22]:
new_spine_with_count_dev_stage =pd.DataFrame({"gid": gid_3, "year": year_3, "quarter": quarter_3, "count": count_3, "lateset_year_development": lateset_year_3})
new_spine_with_count_dev_stage.head()

gid  year  quarter  count  lateset_year_development
0  49182  1970        1      0                         0
1  49182  1970        2      0                         0
2  49182  1970        3      0                         0
3  49182  1970        4      0                         0
4  49182  1980        1      0                         0

In [23]:
new_spine_with_count_dev_stage[new_spine_with_count_dev_stage["gid"]==199878].to_csv("tmp.csv")

In [24]:
new_spine_with_count_dev_stage["lateset_year_development"] = new_spine_with_count_dev_stage["lateset_year_development"].replace({
            0: np.nan, 
            1: 'Grassroots', 2:'Exploration',  3: 'Advanced Exploration',
            4: 'Target Outline', 5: 'Reserves Developmen', 6:'Advanced Exploration', 7:'Prefeas/Scoping',
            8: 'Feasibility',  9: 'Feasibility Started', 10:'Feasibility Complete', 
            11: 'Preproduction', 12: 'Construction Planned', 13:'Construction Started', 
            14: 'Commissioning', 15: 'Operating', 16: 'Satellite' , 
            17: 'Expansion', 18: 'Limited Production', 19: 'Residual Production', 20: 'Closed'
})
new_spine_with_count_dev_stage["count"] = new_spine_with_count_dev_stage["count"].astype("int")
new_spine_with_count_dev_stage = new_spine_with_count_dev_stage.rename(columns={"gid":"PRIO grid id"})

In [25]:
new_spine_with_count_dev_stage.to_csv("../output/snl_quarterly_time_series.csv")

### verification

In [26]:
subset_merge[["PRIO grid id"]==59983]

KeyError: False

In [ ]:
new_spine_with_count_dev_stage[new_spine_with_count_dev_stage["PRIO grid id"]==59983]